In [1]:
import numpy as np
from simulation_framework.item_response_simulation import item_response_simulation
from simulation_framework.simulate_competency import respondent_population
from simulation_framework.simulate_responses import response_simulation
from scipy.stats import multivariate_normal
import models
import em_algorithm
import pandas as pd
import time
from girth import multidimensional_twopl_mml
from girth import twopl_mml
import cma
import scipy
from simulation_experiment import mirt_simulation_experiment

C:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\models
C:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\models


## Experiment 0: MIRT-2PL-Gain Performance Benchmark

In [ ]:
performance_dict = mirt_simulation_experiment(sample_size=100, item_dimension=12, latent_dimension=2, 
                                             q_type="full", stop_threshold=5, person_method="BFGS",
                                            sigma_constraint="unconstrained", real_theta=True)

Real latent covariance: [[1.     0.3461 0.4861 0.1685]
 [0.3461 1.     0.0899 0.2117]
 [0.4861 0.0899 1.     0.1805]
 [0.1685 0.2117 0.1805 1.    ]]
Covariance matrix is good: True
EM Iteration 2
Current Monte Carlo Sample size: 300


c:\Users\Jesper\Anaconda3\lib\site-packages\scipy\stats\_qmc.py:2039: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  samples = self.engine.random(n)


[[1.         0.54027906]
 [0.54027906 1.        ]]
Step: 2: current parameter_diff: 10.773158391210941, current marginal loglikelihood: -602.1963862654185
EM Iteration 3
Current Monte Carlo Sample size: 300
[[1.         0.53554913]
 [0.53554913 1.        ]]


c:\Users\Jesper\Anaconda3\lib\site-packages\scipy\stats\_qmc.py:2039: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  samples = self.engine.random(n)


Step: 3: current parameter_diff: 8.198550278652238, current marginal loglikelihood: -581.5782131187312
EM Iteration 4
Current Monte Carlo Sample size: 330
[[1.         0.50841278]
 [0.50841278 1.        ]]


...

Step: 11: current parameter_diff: 7.148008191757464, current marginal loglikelihood: -545.2349456454795
EM Iteration 12


c:\Users\Jesper\Anaconda3\lib\site-packages\scipy\stats\_qmc.py:2039: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  samples = self.engine.random(n)


Current Monte Carlo Sample size: 529
[[1.     0.5778 0.6421 0.168 ]
 [0.5778 1.     0.     0.0207]
 [0.6421 0.     1.     0.2466]
 [0.168  0.0207 0.2466 1.    ]]


c:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\em_algorithm\e_step_mirt_2pl_gain.py:187: RuntimeWarning: divide by zero encountered in log
  np.log(inv_icc), np.subtract(r_0, r_item))  # TODO: Make this save
c:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\em_algorithm\e_step_mirt_2pl_gain.py:186: RuntimeWarning: invalid value encountered in multiply
  log_likelihood_item = np.multiply(np.log(icc_values), r_item) + np.multiply(
C:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\em_algorithm\m_step_mirt_2pl.py:71: RuntimeWarning: overflow encountered in exp
  population = random.choices(population=population_base, weights=np.exp(


KeyboardInterrupt: 

In [2]:
def print_result(result_dict, description=""):
    #Description
    print("------------------------------------")
    print("##### Results for {0}".format(description))
    #Experiment Properties:
    latent_dimension = result_dict["sample"]["latent_dimension"]
    item_dimension = result_dict["sample"]["item_dimension"]
    sample_size = result_dict["sample"]["sample_size"]
    print("Latent dimension: {0},  Item dimension: {1}, sample size {2} \\".format(latent_dimension, item_dimension, sample_size))
    #Performance/time
    ep_dict = result_dict["early_performance"]
    lp_dict = result_dict["late_performance"]
    early_runtime = np.round(ep_dict["run"]["runtime"], 2)
    early_steps = ep_dict["run"]["number_steps"]
    print("Early runtime: {0} seconds, {1} steps, {2} seconds per step \\".format(early_runtime, early_steps, np.round(early_runtime/early_steps, 2)))
    late_runtime = np.round(lp_dict["run"]["runtime"], 2)
    late_steps = lp_dict["run"]["number_steps"]
    print("Late runtime: {0} seconds, {1} steps, {2} seconds per step \\".format(late_runtime, late_steps, np.round(late_runtime/late_steps, 2)))
    #Performance/results
    # Early Likelihood
    early_l_optimal = np.round(ep_dict["early_marginal_likelihood"]["optimal"], 2)
    early_l_estimated = np.round(ep_dict["early_marginal_likelihood"]["estimated"], 2)
    early_l_real = np.round(ep_dict["early_marginal_likelihood"]["initial"], 2)
    print("Early Marginal Likelihood, Optimal: {0}, Estimated: {1}, Initial {2} \\".format(early_l_optimal,early_l_estimated,early_l_real))
    # Late Likelihood
    late_l_optimal = np.round(lp_dict["late_marginal_likelihood"]["optimal"], 2)
    late_l_estimated = np.round(lp_dict["late_marginal_likelihood"]["estimated"], 2)
    late_l_real = np.round(lp_dict["late_marginal_likelihood"]["initial"], 2)
    print("Late Marginal Likelihood, Optimal: {0}, Estimated: {1}, Initial {2}".format(late_l_optimal,late_l_estimated,late_l_real))
    #print("Performance: rmse-mean = {0} \\".format(np.round(np.mean(np.array(list(result_dict["early_performance"].values()))), 4)))
    rmse_early_model = ep_dict["rmse"]
    
    early_rmse_frame = pd.DataFrame(columns=["rmse_A", "rmse_delta", "rmse_sigma"])
    early_rmse_frame = early_rmse_frame.append(rmse_early_model, ignore_index=True)
    index = ["estimated"]
    for baseline in list(result_dict["baselines"]["early"].keys()):
        rmse_baseline = result_dict["baselines"]["early"][baseline]["performance"]["rmse"]
        early_rmse_frame = early_rmse_frame.append(rmse_baseline, ignore_index=True)
        index.append(baseline)
    early_rmse_frame.index = index
    early_rmse_frame.columns = ["early_A", "early_delta", "early_sigma"]

    rmse_late_model = lp_dict["rmse"]
    
    late_rmse_frame = pd.DataFrame(columns=["rmse_A", "rmse_delta", "rmse_sigma"])
    late_rmse_frame = late_rmse_frame.append(rmse_late_model, ignore_index=True)
    index = ["estimated"]
    for baseline in list(result_dict["baselines"]["late"].keys()):
        rmse_baseline = result_dict["baselines"]["late"][baseline]["performance"]["rmse"]
        late_rmse_frame = late_rmse_frame.append(rmse_baseline, ignore_index=True)
        index.append(baseline)
    late_rmse_frame.index = index
    late_rmse_frame.columns = ["late_A", "late_delta", "late_sigma", "psi"]
    rmse_frame = pd.concat((early_rmse_frame, late_rmse_frame), axis=1)
    print(rmse_frame.to_markdown())
    print("####")
    #Individual 
    rmse_theta = ep_dict["individual"]["rmse"]
    rmse_gain = lp_dict["individual"]["rmse"]
    print("Performance on Individual Level \\")
    print("rmse theta: {0} \\".format(rmse_theta))
    print("rmse gain: {0}".format(rmse_gain))
                                   

In [3]:
print_result(performance_dict, "With unconstrained BFGS and real theta")

NameError: name 'performance_dict' is not defined


------------------------------------
##### Results for Increasing Monte Carlo sample-size based on ttest with normal MC
Latent dimension: 2,  Item dimension: 6, sample size 100 \
Runtime: 3.3 seconds, 15 steps, 0.22 seconds per step \
Optimal marginal Likelihood: -371.46, Estimated: -365.03, Initial -374.57
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     | 0.660667 |     0.316008 |    0.0573464 |
| early_initial | 0.509902 |     0.28936  |    0.212132  |
| girth         | 1.0658   |     0.31617  |    0.12943   |
| early_direct  | 0.848957 |     0.59659  |    0.116858  |

------------------------------------
##### Results for Untested full Model
Latent dimension: 2,  Item dimension: 10, sample size 100 \
Early runtime: 2.56 seconds, 8 steps, 0.32 seconds per step \
Late runtime: 107.71 seconds, 5 steps, 21.54 seconds per step \
Early Marginal Likelihood, Optimal: -470.21, Estimated: -463.83, Initial -527.4 \
Late Marginal Likelihood, Optimal: 3.89, Estimated: 3.92, Initial 3.81
|           |   early_A |   early_delta |   early_sigma |   late_A |   late_delta |   late_sigma |      psi |
|:----------|----------:|--------------:|--------------:|---------:|-------------:|-------------:|---------:|
| estimated |   1.08941 |      0.391419 |     0.0115966 |  1.83043 |      1.33621 |     0.674368 | 0.395368 |
| initial   |   3.07758 |      0.621379 |     0.329017  |  2.74743 |      1.52522 |     0.320885 | 0.210157 |
####
Performance on Individual Level \
rmse theta: 0.864016477222037 \
rmse gain: 0.8736126145557623

------------------------------------
##### Results for Fixed marginal Loglikelihood
Latent dimension: 2,  Item dimension: 10, sample size 100 \
Early runtime: 2.5 seconds, 7 steps, 0.36 seconds per step \
Late runtime: 322.67 seconds, 9 steps, 35.85 seconds per step \
Early Marginal Likelihood, Optimal: -494.37, Estimated: -479.73, Initial -540.27 \
Late Marginal Likelihood, Optimal: -359.97, Estimated: -381.08, Initial -431.02
|           |   early_A |   early_delta |   early_sigma |   late_A |   late_delta |   late_sigma |      psi |
|:----------|----------:|--------------:|--------------:|---------:|-------------:|-------------:|---------:|
| estimated |   1.42983 |      0.837686 |     0.139583  |  4.08714 |      2.46488 |    0.243386  | 0.343469 |
| initial   |   2.6978  |      0.668599 |     0.0490025 |  5.45633 |      1.95892 |    0.0963787 | 0.22315  |
####
Performance on Individual Level \
rmse theta: 0.6755600631466341 \
rmse gain: 780.9161055994681

------------------------------------
##### Results for Genetic Algorithm
Latent dimension: 2,  Item dimension: 10, sample size 100 \
Early runtime: 3.42 seconds, 7 steps, 0.49 seconds per step \
Late runtime: 166.44 seconds, 10 steps, 16.64 seconds per step \
Early Marginal Likelihood, Optimal: -445.8, Estimated: -440.04, Initial -525.38 \
Late Marginal Likelihood, Optimal: -452.07, Estimated: -436.89, Initial -535.83
|           |   early_A |   early_delta |   early_sigma |   late_A |   late_delta |   late_sigma |      psi |
|:----------|----------:|--------------:|--------------:|---------:|-------------:|-------------:|---------:|
| estimated |   3.52076 |      0.390601 |     0.0378639 |  5.26244 |     0.403017 |    0.134472  | 0.317107 |
| initial   |   5.04367 |      0.567904 |     0.0948937 |  4.62289 |     1.22823  |    0.0472347 | 0.344357 |
####
Performance on Individual Level \
rmse theta: 0.8067692103719907 \
rmse gain: 560.8514011989563

------------------------------------
##### Results for With constrained BFGS
Latent dimension: 2,  Item dimension: 10, sample size 100 \
Early runtime: 3.39 seconds, 8 steps, 0.42 seconds per step \
Late runtime: 222.08 seconds, 11 steps, 20.19 seconds per step \
Early Marginal Likelihood, Optimal: -450.38, Estimated: -444.2, Initial -522.62 \
Late Marginal Likelihood, Optimal: -469.55, Estimated: -464.8, Initial -539.91
|           |   early_A |   early_delta |   early_sigma |   late_A |   late_delta |   late_sigma |      psi |
|:----------|----------:|--------------:|--------------:|---------:|-------------:|-------------:|---------:|
| estimated |  0.683488 |      0.604727 |     0.0795495 |  1.19288 |     0.397077 |    0.163978  | 0.367325 |
| initial   |  2.31601  |      0.712129 |     0.0905097 |  2.83422 |     1.04892  |    0.0775696 | 0.226698 |
####
Performance on Individual Level \
rmse theta: 0.7139781405467193 \
rmse gain: 0.8053860461431572

------------------------------------
##### Results for With unconstrained BFGS and save ICC
Latent dimension: 2,  Item dimension: 10, sample size 200 \
Early runtime: 4.94 seconds, 10 steps, 0.49 seconds per step \
Late runtime: 217.77 seconds, 14 steps, 15.56 seconds per step \
Early Marginal Likelihood, Optimal: -1025.93, Estimated: -1014.24, Initial -1143.43 \
Late Marginal Likelihood, Optimal: -1019.85, Estimated: -942.56, Initial -1165.24
|           |   early_A |   early_delta |   early_sigma |   late_A |   late_delta |   late_sigma |     psi |
|:----------|----------:|--------------:|--------------:|---------:|-------------:|-------------:|--------:|
| estimated |   1.28713 |      0.405811 |      0.022486 |  2.34827 |      3.08274 |    0.0799031 | 0.25137 |
| initial   |   2.81054 |      0.442962 |      0.257387 |  4.62677 |      1.05851 |    0.0787717 | 0.33902 |
####
Performance on Individual Level \
rmse theta: 0.8966706033114552 \
rmse gain: 0.8351295524334317

------------------------------------
##### Results for With unconstrained BFGS and save ICC and density
Latent dimension: 2,  Item dimension: 10, sample size 100 \
Early runtime: 3.18 seconds, 7 steps, 0.45 seconds per step \
Late runtime: 138.53 seconds, 15 steps, 9.24 seconds per step \
Early Marginal Likelihood, Optimal: -520.2, Estimated: -510.43, Initial -543.26 \
Late Marginal Likelihood, Optimal: -423.14, Estimated: -395.86, Initial -478.51
|           |   early_A |   early_delta |   early_sigma |   late_A |   late_delta |   late_sigma |       psi |
|:----------|----------:|--------------:|--------------:|---------:|-------------:|-------------:|----------:|
| estimated |   1.46673 |      0.222442 |      0.180454 |  8.89911 |      3.884   |    0.0590434 | 0.0987126 |
| initial   |   1.98992 |      0.373841 |      0.138664 |  3.10531 |      1.01749 |    0.0374059 | 0.279826  |
####
Performance on Individual Level \
rmse theta: 0.823547261347214 \
rmse gain: 1.1151668629591303

------------------------------------
##### Results for With unconstrained BFGS and unsave ICC and save density
Latent dimension: 2,  Item dimension: 12, sample size 100 \
Early runtime: 3.87 seconds, 8 steps, 0.48 seconds per step \
Late runtime: 134.42 seconds, 9 steps, 14.94 seconds per step \
Early Marginal Likelihood, Optimal: -602.6, Estimated: -592.22, Initial -670.75 \
Late Marginal Likelihood, Optimal: -456.88, Estimated: -452.65, Initial -554.35
|           |   early_A |   early_delta |   early_sigma |   late_A |   late_delta |   late_sigma |       psi |
|:----------|----------:|--------------:|--------------:|---------:|-------------:|-------------:|----------:|
| estimated |  0.690325 |      0.546951 |     0.0650538 |  1.27364 |     0.938354 |    0.0915703 | 0.0813927 |
| initial   |  2.45374  |      0.70644  |     0.213122  |  2.99672 |     1.01538  |    0.137037  | 0.289388  |
####
Performance on Individual Level \
rmse theta: 0.9424593402985434 \
rmse gain: 0.9094035651380942

## Experiment 1: MIRT-2PL Parameter Recovery

In [2]:
import datetime

def repeat_mirt_experiment(experiment, repetitions):
    multiple_result_dict = {}
    for i in range(0, repetitions):
        multiple_result_dict[datetime.datetime.now()] = experiment()
    return(multiple_result_dict)

In [3]:
experiment_1_1 = lambda: mirt_simulation_experiment(sample_size=100, item_dimension=12, latent_dimension=2,
                                            q_type="full", stop_threshold=20, person_method="BFGS",
                                            sigma_constraint="unconstrained")
result_dict = repeat_mirt_experiment(experiment_1_1, 2)

Real latent covariance: [[1.     0.     0.     0.1695]
 [0.     1.     0.     0.4911]
 [0.     0.     1.     0.5247]
 [0.1695 0.4911 0.5247 1.    ]]
Covariance matrix is good: True
EM Iteration 2
Current Monte Carlo Sample size: 300
[[1.         0.49079947]
 [0.49079947 1.        ]]


c:\Users\Jesper\Anaconda3\lib\site-packages\scipy\stats\_qmc.py:2039: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  samples = self.engine.random(n)


Step: 2: current parameter_diff: 12.927111945207757, current marginal loglikelihood: -592.5073648135902
EM Iteration 3
Current Monte Carlo Sample size: 300
[[1.         0.40213035]
 [0.40213035 1.        ]]
Step: 3: current parameter_diff: 9.505778576633883, current marginal loglikelihood: -572.9378901363236
EM Iteration 4
Current Monte Carlo Sample size: 330
[[1.         0.28442727]
 [0.28442727 1.        ]]


c:\Users\Jesper\Anaconda3\lib\site-packages\scipy\stats\_qmc.py:2039: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  samples = self.engine.random(n)


Step: 4: current parameter_diff: 7.6368472781001975, current marginal loglikelihood: -566.3290481355793
EM Iteration 5
Current Monte Carlo Sample size: 363
[[1.         0.18364482]
 [0.18364482 1.        ]]
Step: 5: current parameter_diff: 5.567487611924966, current marginal loglikelihood: -563.6173541505666
EM Iteration 2


...

Step: 5: current parameter_diff: 8.736992759031816, current marginal loglikelihood: -513.6672899638598
EM Iteration 6


c:\Users\Jesper\Anaconda3\lib\site-packages\scipy\stats\_qmc.py:2039: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  samples = self.engine.random(n)


Current Monte Carlo Sample size: 363
[[1.         0.54880351 0.62967172 0.3653768 ]
 [0.54880351 1.         0.46632738 0.40804193]
 [0.62967172 0.46632738 1.         0.4661568 ]
 [0.3653768  0.40804193 0.4661568  1.        ]]


c:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\em_algorithm\e_step_mirt_2pl_gain.py:187: RuntimeWarning: divide by zero encountered in log
  np.log(inv_icc), np.subtract(r_0, r_item))  # TODO: Make this save
c:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\em_algorithm\e_step_mirt_2pl_gain.py:186: RuntimeWarning: invalid value encountered in multiply
  log_likelihood_item = np.multiply(np.log(icc_values), r_item) + np.multiply(
C:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\em_algorithm\m_step_mirt_2pl.py:71: RuntimeWarning: overflow encountered in exp
  population = random.choices(population=population_base, weights=np.exp(


Step: 6: current parameter_diff: 7.9713474866737455, current marginal loglikelihood: -510.9760647269765
Absolute diff in A:
[[0.         0.11033676]
 [0.         0.55785308]
 [0.         0.61440032]
 [0.         0.15167896]
 [0.         1.43969578]
 [0.40490474 0.        ]
 [6.78782412 0.        ]
 [0.16677298 0.        ]
 [5.1172315  0.77051977]
 [1.00975638 0.23122067]
 [0.02624702 0.15338498]
 [1.12657888 0.14060907]]
Absolute diff in delta:
[0.43746994 0.3471241  0.34525609 0.21184708 0.39802006 0.24719617
 0.44683922 0.30867422 1.09947736 1.83778328 0.55601312 0.52897869]
Absolute diff in sigma:
[[0.00e+00 6.77e-02]
 [6.77e-02 1.00e-04]]
Absolute diff in A:
[[1.39765626e-01 1.07192960e+00]
 [1.65767735e-01 2.33739516e+00]
 [2.33698375e+00 2.41625406e-01]
 [2.36199724e+00 2.91604596e+00]
 [8.49769907e-03 7.61170826e-01]
 [3.15776253e+00 3.08144968e+00]
 [3.69249552e+00 8.63656653e+00]
 [4.58756084e-01 2.35553400e-01]
 [2.30906878e+00 1.25867043e+00]
 [9.22801789e-01 1.71522155e-01]

In [4]:
result_dict.keys()

dict_keys([datetime.datetime(2022, 10, 29, 18, 37, 46, 222541), datetime.datetime(2022, 10, 29, 18, 38, 56, 273525)])

In [26]:
def parse_method_dict(performance_dict):
        ep_dict = performance_dict["early_performance"]
        lp_dict = performance_dict["late_performance"]
        early_runtime = np.round(ep_dict["run"]["runtime"], 2)
        early_steps = ep_dict["run"]["number_steps"]
        late_runtime = np.round(lp_dict["run"]["runtime"], 2)
        late_steps = lp_dict["run"]["number_steps"]
        early_l_optimal = np.round(ep_dict["early_marginal_likelihood"]["optimal"], 2)
        early_l_estimated = np.round(ep_dict["early_marginal_likelihood"]["estimated"], 2)
        late_l_optimal = np.round(lp_dict["late_marginal_likelihood"]["optimal"], 2)
        late_l_estimated = np.round(lp_dict["late_marginal_likelihood"]["estimated"], 2)
        rmse_early_model = ep_dict["rmse"]
        rmse_late_model = lp_dict["rmse"]
        rmse_early_A = rmse_early_model["rmse_A"]
        rmse_early_delta = rmse_early_model["rmse_delta"]
        rmse_early_sigma = rmse_early_model["rmse_sigma"]
        rmse_late_A = rmse_late_model["rmse_A"]
        rmse_late_delta = rmse_late_model["rmse_delta"]
        rmse_late_psi = rmse_late_model["rmse_psi"]
        rmse_late_sigma = rmse_late_model["rmse_sigma"]
        performance_vector = np.array([early_runtime, late_runtime, early_steps, late_steps, early_l_optimal, early_l_estimated,
                                 late_l_optimal, late_l_estimated,
                                rmse_early_A, rmse_early_delta, rmse_early_sigma, 
                                rmse_late_A, rmse_late_delta, rmse_late_psi, rmse_late_sigma])
        performance_df = pd.DataFrame(data=pd.DataFrame(performance_vector).transpose())
        performance_df.columns = ["early_runtime", "late_runtime", "early_steps", "late_steps", "early_l_optimal", "early_l_estimated",
                                 "late_l_optimal", "late_l_estimated", 
                                "rmse_early_A", "rmse_early_delta", "rmse_early_sigma", 
                                "rmse_late_A", "rmse_late_delta", "rmse_psi", "rmse_late_sigma"]
        return(performance_df)

def get_result_df(_result_dict):
    keylist = list(_result_dict.keys())
    latent_dimension = _result_dict[keylist[0]]["sample"]["latent_dimension"]
    item_dimension = _result_dict[keylist[0]]["sample"]["item_dimension"]
    sample_size = _result_dict[keylist[0]]["sample"]["sample_size"]
    meta = {"latent_dimension": latent_dimension, "item_dimension": item_dimension, "sample_size": sample_size}
    result_df = pd.DataFrame()
    for key in keylist:
        result_dict = _result_dict[key]
        method_list = list(result_dict.keys())
        method_list.remove("sample")
        for method in method_list:
            method_info_vector = np.array([latent_dimension, item_dimension, sample_size, key, method])
            method_info_df = pd.DataFrame(data=pd.DataFrame(method_info_vector).transpose())
            method_info_df.columns=["latent_dimension", "item_dimension", "sample_size", "key", "method"]
            performance_dict = result_dict[method]
            method_row = parse_method_dict(performance_dict)
            method_row = pd.concat((method_info_df, method_row), axis=1)
            result_df = pd.concat((result_df, method_row), axis=0)
    return(result_df)


In [29]:
get_result_df(result_dict).groupby(["method"]).mean()

,early_runtime,late_runtime,early_steps,late_steps,early_l_optimal,early_l_estimated,late_l_optimal,late_l_estimated,rmse_early_A,rmse_early_delta,rmse_early_sigma,rmse_late_A,rmse_late_delta,rmse_psi,rmse_late_sigma
method,,,,,,,,,,,,,,,
initial,0.000,0.000,0.0,0.0,-584.875,-659.09,-579.265,-621.66,3.044415,0.743931,0.246002,3.180891,1.768273,0.372834,0.069014
late_em,2.525,53.135,6.0,7.0,-585.600,-577.00,-536.800,-524.17,1.514053,0.750542,0.088848,3.203046,1.849611,0.355705,0.109496


In [23]:
#Univariate IRT Recovery
result_dict = mirt_param_recovery(1000, q_type="singular", item_dimension=20, latent_dimension=1, stop_threshold=3)

Real latent covariance: [[1.]]
Covariance matrix is good: True
EM Iteration 2


c:\Users\Jesper\Anaconda3\lib\site-packages\scipy\stats\_qmc.py:2039: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  samples = self.engine.random(n)


Current Monte Carlo Sample size: 150
Step: 2: current parameter_diff: 22.742099580485075, current marginal loglikelihood: -8816.976449557947
EM Iteration 3
Current Monte Carlo Sample size: 165
Step: 3: current parameter_diff: 10.719114330591957, current marginal loglikelihood: -8747.334230330345
EM Iteration 4
Current Monte Carlo Sample size: 165
Step: 4: current parameter_diff: 4.741911411251834, current marginal loglikelihood: -8715.022616878563
EM Iteration 5
Current Monte Carlo Sample size: 165
Step: 5: current parameter_diff: 1.8142332751245616, current marginal loglikelihood: -8712.134520806754
EM Iteration 6
Current Monte Carlo Sample size: 181
Step: 6: current parameter_diff: 1.2343984015305836, current marginal loglikelihood: -8716.601796257768
EM Iteration 7
Current Monte Carlo Sample size: 199
Step: 7: current parameter_diff: 1.691136460449655, current marginal loglikelihood: -8705.167502668424
EM Iteration 8
Current Monte Carlo Sample size: 199
Step: 8: current parameter_di

In [3]:
print_result(result_dict, "Monte Carlo sample varying through ttest with normal MC")

NameError: name 'print_result' is not defined

### Univariate IRT/Full GA

------------------------------------
##### Item Dimension=12, sample_size=200: Results for Fixed sample per em-step
Runtime: 4.38 seconds, 7 steps, 0.63 seconds per step \
Optimal marginal Likelihood: -1284.1, Estimated: -1273.09, Initial -1330.43
|           |   rmse_A |   rmse_delta |   rmse_sigma |
|:----------|---------:|-------------:|-------------:|
| Estimated | 0.508687 |     0.196944 |            0 |
| Girth     | 0.377882 |     2.65875  |            0 |
| direct    | 0.305489 |     0.318168 |            0 |
| Initial   | 1.5583   |     0.861273 |            0 |

------------------------------------
##### Item Dimension=20, sample_size=1000: Results for Fixed sample per em-step
Runtime: 85.11 seconds, 19 steps, 4.48 seconds per step \
Optimal marginal Likelihood: -9199.01, Estimated: -9183.03, Initial -9784.69
|           |   rmse_A |   rmse_delta |   rmse_sigma |
|:----------|---------:|-------------:|-------------:|
| Estimated | 0.352003 |     0.578688 |            0 |
| Girth     | 0.468473 |     4.98511  |            0 |
| Initial   | 1.66615  |     2.01114  |            0 |

------------------------------------
##### Results for Monte Carlo sample varying through ttest
Latent dimension: 1,  Item dimension: 20, sample size 1000 \
Runtime: 31.86 seconds, 10 steps, 3.19 seconds per step \
Optimal marginal Likelihood: -9619.61, Estimated: -9601.49, Initial -9970.74
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     | 0.175897 |     0.223543 |            0 |
| early_initial | 1.21508  |     1.78456  |            0 |
| girth         | 0.327131 |     4.71968  |            0 |

------------------------------------
##### Results for Monte Carlo sample varying through ttest with normal MC
Latent dimension: 1,  Item dimension: 20, sample size 1000 \
Runtime: 37.86 seconds, 20 steps, 1.89 seconds per step \
Optimal marginal Likelihood: -8714.19, Estimated: -8697.21, Initial -9512.15
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     | 0.182642 |     0.211132 |            0 |
| early_initial | 1.69549  |     1.49975  |            0 |
| girth         | 0.313235 |     4.2295   |            0 |

In [12]:
performance_dict = mirt_param_recovery(sample_size=1000, item_dimension=20, latent_dimension=2, q_type="seperated", girth=False, stop_threshold=1)

Real latent covariance: [[1.     0.1921]
 [0.1921 1.    ]]
Covariance matrix is good: True
EM Iteration 2


c:\Users\Jesper\Anaconda3\lib\site-packages\scipy\stats\_qmc.py:2039: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  samples = self.engine.random(n)


Current Monte Carlo Sample size: 300
Step: 2: current parameter_diff: 9.087457477954748, current marginal loglikelihood: -12789.826209045792
EM Iteration 3
Current Monte Carlo Sample size: 300
Step: 3: current parameter_diff: 9.347286786290436, current marginal loglikelihood: -12431.792159340152
EM Iteration 4
Current Monte Carlo Sample size: 300
Step: 4: current parameter_diff: 5.994195183510891, current marginal loglikelihood: -12324.415808814525
EM Iteration 5
Current Monte Carlo Sample size: 330
Step: 5: current parameter_diff: 3.466965558217997, current marginal loglikelihood: -12291.666326556195
EM Iteration 6
Current Monte Carlo Sample size: 363
Step: 6: current parameter_diff: 3.0517259450824583, current marginal loglikelihood: -12274.293408739399
EM Iteration 7
Current Monte Carlo Sample size: 399
Step: 7: current parameter_diff: 1.6828800520798761, current marginal loglikelihood: -12268.44295499595
EM Iteration 8
Current Monte Carlo Sample size: 438
Step: 8: current parameter

c:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\em_algorithm\e_step_mirt_2pl.py:264: RuntimeWarning: divide by zero encountered in log
  icc_values), r_item_theta) + np.multiply(np.log(1-icc_values), np.subtract(r_0_theta, r_item_theta))


Step: 13: current parameter_diff: 0.40565533319178126, current marginal loglikelihood: -12248.475980359144
EM Iteration 14
Current Monte Carlo Sample size: 581
Step: 14: current parameter_diff: 0.3981727860081429, current marginal loglikelihood: -12251.628743992456
EM Iteration 15
Current Monte Carlo Sample size: 639
Step: 15: current parameter_diff: 0.3759899463682548, current marginal loglikelihood: -12251.09622265484
EM Iteration 16
Current Monte Carlo Sample size: 702
Step: 16: current parameter_diff: 0.07730277535592268, current marginal loglikelihood: -12250.114423241039
EM Iteration 17
Current Monte Carlo Sample size: 772
Step: 17: current parameter_diff: 0.35397135818851444, current marginal loglikelihood: -12249.62312895641
Absolute diff in A:
[[ 0.14141196  0.        ]
 [ 0.14652275  0.        ]
 [ 0.24777834  0.        ]
 [ 0.39441549  0.        ]
 [ 0.12016762  0.        ]
 [ 0.32373102  0.        ]
 [ 2.04123377  0.        ]
 [ 0.07098307  0.        ]
 [ 0.06713308  0.    

In [13]:
print_result(performance_dict, "Fixed Issue with simulation Covariance")

------------------------------------
##### Results for Fixed Issue with simulation Covariance
Latent dimension: 2,  Item dimension: 20, sample size 1000 \
Runtime: 51.07 seconds, 18 steps, 2.84 seconds per step \
Optimal marginal Likelihood: -11670.7, Estimated: -12247.32, Initial -13560.2
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  1.78631 |     0.262764 |    0.0418607 |
| early_initial |  2.54062 |     0.826317 |    0.217718  |


------------------------------------
##### Results for Seperated Q-Matrix
Latent dimension: 2,  Item dimension: 20, sample size 1000 \
Runtime: 142.07 seconds, 26 steps, 5.46 seconds per step \
Optimal marginal Likelihood: -9384.38, Estimated: -10189.47, Initial -11459.9
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  4.00013 |      3.10678 |    0.0201383 |
| early_initial |  4.5183  |      2.5846  |    0.165491  |

------------------------------------
##### Results for BFGS for Q_0 with approximate Jacobi
Latent dimension: 2,  Item dimension: 20, sample size 1000 \
Runtime: 61.55 seconds, 12 steps, 5.13 seconds per step \
Optimal marginal Likelihood: -9363.89, Estimated: -9711.58, Initial -10528.79
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  3.02507 |      1.83482 |    0.0645047 |
| early_initial |  2.06432 |      2.18337 |    0.289049  |

------------------------------------
##### Results for Monte Carlo sample varying through ttest
Latent dimension: 2,  Item dimension: 20, sample size 1000 \
Runtime: 325.73 seconds, 42 steps, 7.76 seconds per step \
Optimal marginal Likelihood: -9929.77, Estimated: -10179.96, Initial -10767.41
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  1.25777 |     0.965894 |   0.00557421 |
| early_initial |  1.98812 |     1.68516  |   0.719705   |

------------------------------------
##### Results for Fixed random Covariance 1, girth included
Latent dimension: 2,  Item dimension: 20, sample size 1000 \
Runtime: 57.16 seconds, 14 steps, 4.08 seconds per step \
Optimal marginal Likelihood: -11668.66, Estimated: -11672.6, Initial -12307.41
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     | 0.183251 |    0.0861252 |    0.0215108 |
| early_initial | 0.786712 |    0.393667  |    0.0471788 |
| girth         | 1.28298  |    0.263374  |    0.350541  |

------------------------------------
##### Results for Fixed Issue with simulation Covariance
Latent dimension: 2,  Item dimension: 20, sample size 1000 \
Runtime: 24.71 seconds, 11 steps, 2.25 seconds per step \
Optimal marginal Likelihood: -11186.38, Estimated: -11389.57, Initial -12010.77
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  1.83766 |      1.4664  |    0.0743169 |
| early_initial |  2.7041  |      2.66224 |    0.0627911 |

In [16]:
performance_dict = mirt_param_recovery(sample_size=1000, item_dimension=20, latent_dimension=3, q_type="pyramid", girth=False, stop_threshold=10, ensure_id=True, q_share=0.6)

Real latent covariance: [[1.     0.5392 0.4093]
 [0.5392 1.     0.5406]
 [0.4093 0.5406 1.    ]]
Covariance matrix is good: True
EM Iteration 2


c:\Users\Jesper\Anaconda3\lib\site-packages\scipy\stats\_qmc.py:2039: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  samples = self.engine.random(n)


Current Monte Carlo Sample size: 600


c:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\em_algorithm\e_step_mirt_2pl.py:198: RuntimeWarning: divide by zero encountered in log
  factor = np.log(self.model.latent_density(theta, sigma=sigma))
c:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\em_algorithm\e_step_mirt_2pl.py:198: RuntimeWarning: divide by zero encountered in log
  factor = np.log(self.model.latent_density(theta, sigma=sigma))


[[1.         0.22255116 0.4851635 ]
 [0.22255116 1.         0.28710379]
 [0.4851635  0.28710379 1.        ]]
Step: 2: current parameter_diff: 22.836966115835242, current marginal loglikelihood: -11263.404209075177
EM Iteration 3


...

c:\Users\Jesper\Anaconda3\lib\site-packages\scipy\stats\_qmc.py:2039: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  samples = self.engine.random(n)


Current Monte Carlo Sample size: 877
[[ 1.         -0.18130942  0.49736388]
 [-0.18130942  1.         -0.04688012]
 [ 0.49736388 -0.04688012  1.        ]]
Step: 7: current parameter_diff: 4.21590743744123, current marginal loglikelihood: -10830.644137405307
EM Iteration 8


c:\Users\Jesper\Anaconda3\lib\site-packages\scipy\stats\_qmc.py:2039: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  samples = self.engine.random(n)


Current Monte Carlo Sample size: 964
[[ 1.         -0.15152568  0.50240847]
 [-0.15152568  1.         -0.07665482]
 [ 0.50240847 -0.07665482  1.        ]]


c:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\em_algorithm\e_step_mirt_2pl.py:264: RuntimeWarning: divide by zero encountered in log
  icc_values), r_item_theta) + np.multiply(np.log(1-icc_values), np.subtract(r_0_theta, r_item_theta))
c:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\em_algorithm\e_step_mirt_2pl.py:264: RuntimeWarning: invalid value encountered in multiply
  icc_values), r_item_theta) + np.multiply(np.log(1-icc_values), np.subtract(r_0_theta, r_item_theta))
c:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\em_algorithm\m_step_mirt_2pl.py:70: RuntimeWarning: overflow encountered in exp
  population = random.choices(population=population_base, weights=np.exp(


In [8]:
print_result(performance_dict, "With Covariance Matrix partially aligned to Q-matrix")

NameError: name 'performance_dict' is not defined

------------------------------------
##### Results for Pyramid Q-Matrix
Latent dimension: 3,  Item dimension: 20, sample size 200 \
Runtime: 121.22 seconds, 46 steps, 2.64 seconds per step \
Optimal marginal Likelihood: -2144.15, Estimated: -2155.08, Initial -2699.63
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  11.0034 |      1.77461 |     0.889828 |
| early_initial |  10.5116 |      2.15986 |     0.522199 |

------------------------------------
##### Results for Pyramid Q-Matrix
Latent dimension: 3,  Item dimension: 20, sample size 1000 \
Runtime: 313.22 seconds, 51 steps, 6.14 seconds per step \
Optimal marginal Likelihood: -9452.24, Estimated: -9813.87, Initial -11216.48
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  5.34115 |      1.4068  |     0.425982 |
| early_initial |  5.98238 |      2.25109 |     0.234288 |

------------------------------------
##### Results for Pyramid Q-Matrix
Latent dimension: 3,  Item dimension: 30, sample size 2000 \
Runtime: 1521.51 seconds, 76 steps, 20.02 seconds per step \
Optimal marginal Likelihood: -30844.13, Estimated: -32521.34, Initial -37390.6
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  2.55959 |       1.9952 |    0.314331  |
| early_initial |  3.32034 |       3.514  |    0.0239562 |

------------------------------------
##### Results for BFGS for Q_0 with approximated gradient
Latent dimension: 3,  Item dimension: 20, sample size 1000 \
Runtime: 350.28 seconds, 64 steps, 5.47 seconds per step \
Optimal marginal Likelihood: -10498.87, Estimated: -11303.0, Initial -12740.14
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  10.2704 |     0.68597  |    0.318713  |
| early_initial |  10.4626 |     0.846172 |    0.0895367 |

------------------------------------
##### Results for BFGS for Q_0 with exact gradient
Latent dimension: 3,  Item dimension: 20, sample size 1000 \
Runtime: 152.91 seconds, 31 steps, 4.93 seconds per step \
Optimal marginal Likelihood: -11106.69, Estimated: -11211.51, Initial -12370.23
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  3.5074  |      2.54509 |     0.21123  |
| early_initial |  3.51819 |      2.66313 |     0.319647 |

------------------------------------
##### Results for Newton-Raphson for Q_0 with approximated second derivative
Latent dimension: 3,  Item dimension: 20, sample size 1000 \
Runtime: 352.93 seconds, 62 steps, 5.69 seconds per step \
Optimal marginal Likelihood: -10237.22, Estimated: -10482.36, Initial -12569.49
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  5.23694 |      2.45127 |     0.508499 |
| early_initial |  5.36172 |      3.29141 |     0.640209 |

In [13]:
performance_dict = mirt_param_recovery(sample_size=1000, item_dimension=30, latent_dimension=3, q_type="full", girth=False, stop_threshold=10, q_share=0.0)

Real latent covariance: [[1.     0.     0.2545]
 [0.     1.     0.3721]
 [0.2545 0.3721 1.    ]]
Covariance matrix is good: True
EM Iteration 2


c:\Users\Jesper\Anaconda3\lib\site-packages\scipy\stats\_qmc.py:2039: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  samples = self.engine.random(n)


Current Monte Carlo Sample size: 600


c:\Users\Jesper\Anaconda3\lib\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


[[nan nan nan]
 [nan nan nan]
 [nan nan nan]]
[[nan nan nan]
 [nan nan nan]
 [nan nan nan]]
Invalid Covariance encountered, trying last step covariance
Step: 2: current parameter_diff: 41.855085339761395, current marginal loglikelihood: -12550.72345228223
EM Iteration 3
Current Monte Carlo Sample size: 660
[[nan nan nan]
 [nan nan nan]
 [nan nan nan]]
[[nan nan nan]
 [nan nan nan]
 [nan nan nan]]
Invalid Covariance encountered, trying last step covariance
Step: 3: current parameter_diff: 28.922785136715092, current marginal loglikelihood: -12131.893852917503
EM Iteration 4
Current Monte Carlo Sample size: 660
[[ 1.00000000e+000 -7.56483606e-249 -7.12199634e-243]
 [-7.56483606e-249  1.00000000e+000 -1.46994029e-260]
 [-7.12199634e-243 -1.46994029e-260  1.00000000e+000]]


c:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\em_algorithm\e_step_mirt_2pl.py:264: RuntimeWarning: divide by zero encountered in log
  icc_values), r_item_theta) + np.multiply(np.log(1-icc_values), np.subtract(r_0_theta, r_item_theta))
c:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\em_algorithm\e_step_mirt_2pl.py:264: RuntimeWarning: invalid value encountered in multiply
  icc_values), r_item_theta) + np.multiply(np.log(1-icc_values), np.subtract(r_0_theta, r_item_theta))


Step: 4: current parameter_diff: 24.909956480277785, current marginal loglikelihood: -11869.365012663717
EM Iteration 5
Current Monte Carlo Sample size: 660
[[1.         0.07473132 0.19983603]
 [0.07473132 1.         0.24058055]
 [0.19983603 0.24058055 1.        ]]
Step: 5: current parameter_diff: 20.731814136925763, current marginal loglikelihood: -11739.105149945846
EM Iteration 6
Current Monte Carlo Sample size: 660
[[1.         0.02395303 0.25925938]
 [0.02395303 1.         0.26711545]
 [0.25925938 0.26711545 1.        ]]
Step: 6: current parameter_diff: 6.423400003338906, current marginal loglikelihood: -11720.71525563623
EM Iteration 7
Current Monte Carlo Sample size: 726
[[1.         0.02766565 0.2770886 ]
 [0.02766565 1.         0.30801604]
 [0.2770886  0.30801604 1.        ]]
Step: 7: current parameter_diff: 5.599935651052099, current marginal loglikelihood: -11726.261698793634
EM Iteration 8
Current Monte Carlo Sample size: 798
[[1.         0.03611896 0.29298871]
 [0.03611896

c:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\em_algorithm\m_step_mirt_2pl.py:70: RuntimeWarning: overflow encountered in exp
  population = random.choices(population=population_base, weights=np.exp(


Step: 8: current parameter_diff: 8.88763105930743, current marginal loglikelihood: -11739.130271363902
EM Iteration 9
Current Monte Carlo Sample size: 877
[[1.         0.04450944 0.2860444 ]
 [0.04450944 1.         0.34325432]
 [0.2860444  0.34325432 1.        ]]
Step: 9: current parameter_diff: 6.972349751947515, current marginal loglikelihood: -11721.612152810394
EM Iteration 10
Current Monte Carlo Sample size: 964
[[1.         0.04267512 0.28987184]
 [0.04267512 1.         0.33339445]
 [0.28987184 0.33339445 1.        ]]
Step: 10: current parameter_diff: 13.349635834760017, current marginal loglikelihood: -11712.217814439908
EM Iteration 11
Current Monte Carlo Sample size: 1060
[[1.         0.05402791 0.27611641]
 [0.05402791 1.         0.32730623]
 [0.27611641 0.32730623 1.        ]]
Step: 11: current parameter_diff: 13.121900001706786, current marginal loglikelihood: -11752.478352914124
EM Iteration 12
Current Monte Carlo Sample size: 1166
[[1.         0.03472167 0.29289335]
 [0.0

KeyboardInterrupt: 

In [12]:
print_result(performance_dict, "With fixed Discrimination sampling and BFGS")

------------------------------------
##### Results for With fixed Discrimination sampling and BFGS
Latent dimension: 3,  Item dimension: 20, sample size 1000 \
Runtime: 63.64 seconds, 14 steps, 4.55 seconds per step \
Optimal marginal Likelihood: -8684.81, Estimated: -8676.2, Initial -9809.83
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     | 0.933735 |     0.514423 |    0.0383988 |
| early_initial | 2.67536  |     0.636687 |    0.231996  |


------------------------------------
##### Results for With BFGS as covariance optimization Algorithm
Latent dimension: 3,  Item dimension: 20, sample size 1000 \
Runtime: 160.7 seconds, 20 steps, 8.04 seconds per step \
Optimal marginal Likelihood: -11451.44, Estimated: -11746.37, Initial -12757.23
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  4.12413 |      1.19297 |     0.539132 |
| early_initial |  4.06187 |      1.22103 |     0.159225 |

------------------------------------
##### Results for With Cholesky covariance decomposition
Latent dimension: 3,  Item dimension: 20, sample size 1000 \
Runtime: 60.82 seconds, 14 steps, 4.34 seconds per step \
Optimal marginal Likelihood: -11369.92, Estimated: -12102.54, Initial -13629.98
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  2.47435 |     0.400838 |     0.181127 |
| early_initial |  2.98819 |     0.686696 |     0.142997 |

------------------------------------
##### Results for With Cholesky covariance decomposition and BFGS
Latent dimension: 3,  Item dimension: 30, sample size 1000 \
Runtime: 94.77 seconds, 12 steps, 7.9 seconds per step \
Optimal marginal Likelihood: -15960.47, Estimated: -17344.15, Initial -19786.45
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  48.8803 |     0.777924 |     0.149531 |
| early_initial |  49.0831 |     1.33755  |     0.207159 |

------------------------------------
##### Results for With fixed Discrimination sampling and BFGS
Latent dimension: 3,  Item dimension: 30, sample size 1000 \
Runtime: 91.68 seconds, 12 steps, 7.64 seconds per step \
Optimal marginal Likelihood: -12149.74, Estimated: -12157.89, Initial -14965.74
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  0.3929  |     0.249323 |    0.0275205 |
| early_initial |  2.20296 |     0.582838 |    0.380006  |

## Experiment 2: Competency Gain Prediction